In [162]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
 
df = pd.read_excel('MVO Assignment.xlsx')
tickers = df['Unnamed: 1'].dropna()
param = list(df['Unnamed: 5'].dropna())
weight_client  = df['Unnamed: 2'].dropna()

rf = param[3]
rf_per = rf/12

print(tickers)
print(weight_client)
print(param)

# client_tickers = []
# client_weights = []

# ticker = 'dummy'
# weight = 'dummy'


# while ticker != '':
#     ticker = input("Please enter tickers of all stocks in your portfolio, press enter/return to exit:")
#     client_tickers.append(ticker)


# while weight != '':
#     weight = input("Please enter weights of all your stocks, press enter/return to exit:" )
#     client_weights.append(weight)

# print(client_tickers)



1    AAPL
2     IBM
3     MCD
4     PFE
5     WMT
Name: Unnamed: 1, dtype: object
1    0.2
2    0.2
3    0.2
4    0.2
5    0.2
Name: Unnamed: 2, dtype: float64
[datetime.datetime(2015, 11, 1, 0, 0), datetime.datetime(2023, 1, 1, 0, 0), '1mo', 0.03]


In [165]:
import yfinance as yf

history = pd.DataFrame()

for ticker in tickers:
    yf_ticker = yf.Ticker(ticker)
    #history[ticker] = yf_ticker.history(period=param[0], interval = param[1], auto_adjust = True)['Close']
    history[ticker] = yf_ticker.history(interval=param[2], start = param[0],end=param[1], auto_adjust = True)['Close']

    col_shift = ticker + '_shifted'
    history[col_shift] = history[ticker].shift(periods = 1,fill_value = 0, axis = 0)

    col_ret = ticker + '_returns'
    history[col_ret] = (history[ticker] - history[col_shift])/history[col_shift]

    history = history.drop(columns=[col_shift, ticker])


history = history.tail(-1)
mu_vector = []
for i,ticker in enumerate(tickers):
    col_ret = ticker + '_returns'
    mean = history[col_ret].mean()
    col_de_mean = ticker + '_de_meaned'
    history[col_de_mean] = history[col_ret] - mean
    #history = history.drop(columns=[col_ret])

    mu_vector.append(mean) 
    
history.index = history.index.tz_localize(None)

print(mu_vector)
print(history)



[0.022379787245394127, 0.006997668776951506, 0.013322042533608688, 0.011170447770784695, 0.01365863588846816]
            AAPL_returns  IBM_returns  MCD_returns  PFE_returns  WMT_returns  \
Date                                                                           
2015-12-01     -0.106420    -0.003651     0.042977    -0.007002     0.041808   
2016-01-01     -0.075242    -0.093228     0.047740    -0.055452     0.091612   
2016-02-01     -0.006677     0.050004    -0.053240    -0.026894    -0.000301   
2016-03-01      0.133327     0.167649     0.080571     0.009032     0.032409   
2016-04-01     -0.139921    -0.036382     0.006445     0.103576    -0.016425   
...                  ...          ...          ...          ...          ...   
2022-08-01     -0.032552    -0.017891    -0.042108    -0.097585     0.003786   
2022-09-01     -0.119756    -0.063395    -0.080375    -0.032501    -0.017240   
2022-10-01      0.109552     0.163959     0.181676     0.063757     0.097379   
2022-11-01

In [172]:
import openpyxl
import math
import numpy as np
import scipy.stats 

wb = openpyxl.load_workbook('MVO Assignment.xlsx', data_only=True)
sheet = wb['Report']

sheet.cell(column=1, row=2).value = 'Client Portfolio Stocks:'
sheet.cell(column=1, row=3).value = 'Stock Average Return:'
sheet.cell(column=1, row=4).value = 'Stock Standard Deviation:'
sheet.cell(column=1, row=5).value = 'Stock Kurtosis:'
sheet.cell(column=1, row=6).value = 'Stock Skew:'


for i,ticker in enumerate(tickers):
    col_ret = ticker + '_returns'

    sheet.cell(column=i+2, row=2).value = ticker
    sheet.cell(column=i+2, row=3).value = np.mean(history[col_ret])
    sheet.cell(column=i+2, row=4).value = np.std(history[col_ret])
    sheet.cell(column=i+2, row=5).value = scipy.stats.kurtosis (history[col_ret])
    sheet.cell(column=i+2, row=6).value = scipy.stats.skew (history[col_ret])
 
    history = history.drop(columns=[col_ret])


wb.save('MVO Assignment.xlsx')

In [173]:
import numpy as np

# (Epsilon transpose * Epsilon )/ (n-1)
covariance = np.matmul(np.transpose(history),history)/(len(history) - 1)

covariance

,0,1,2,3,4
AAPL_de_meaned,0.007576,0.002006,0.001559,0.001370,0.001556
IBM_de_meaned,0.002006,0.005065,0.001549,0.001740,0.000676
MCD_de_meaned,0.001559,0.001549,0.002598,0.001050,0.001239
PFE_de_meaned,0.001370,0.001740,0.001050,0.004428,0.000874
WMT_de_meaned,0.001556,0.000676,0.001239,0.000874,0.002950


In [182]:
#Question 1

import math
weight_client_mat_transpose = np.asmatrix(weight_client)
covariance = np.asmatrix(covariance)
weight_client_mat = np.asmatrix(np.transpose(weight_client_mat_transpose))


current_client_stddev = math.sqrt(np.matmul(np.matmul(weight_client_mat_transpose,covariance), weight_client_mat))
current_client_exp_ret = np.matmul(mu_vector,weight_client_mat)
current_client_sharpe_ratio = (current_client_exp_ret-rf_per)/current_client_stddev

current_client_stddev_ann = current_client_stddev * math.sqrt(12)
current_client_exp_ret_ann = 12 * current_client_exp_ret
current_client_sharpe_ratio_ann = (current_client_exp_ret_ann-rf)/current_client_stddev_ann

#output to excel

sheet.cell(column=1, row=8).value = 'Current client portfolio metrics:'
sheet.cell(column=2, row=8).value = 'Periodic'
sheet.cell(column=3, row=8).value = 'Annual'

sheet.cell(column=1, row=9).value = 'Expected Returns:'
sheet.cell(column=1, row=10).value = 'Annualized Standard Deviation'
sheet.cell(column=1, row=11).value = 'Annualized Sharpe Ratio'

sheet.cell(column=2, row=9).value = float(current_client_exp_ret)
sheet.cell(column=2, row=10).value = float(current_client_stddev)
sheet.cell(column=2, row=11).value = float(current_client_sharpe_ratio)

sheet.cell(column=3, row=9).value = float(current_client_exp_ret_ann)
sheet.cell(column=3, row=10).value = float(current_client_stddev_ann)
sheet.cell(column=3, row=11).value = float(current_client_sharpe_ratio_ann)



wb.save('MVO Assignment.xlsx')



In [189]:
#Question 2a

a_mat = 2*covariance

a_mat_arr = np.array(a_mat)

num_rows, num_cols = a_mat_arr.shape

new_column = np.ones((num_rows, 1), dtype=int)
a_mat_arr = np.hstack((a_mat_arr, new_column))

new_row = np.ones((1, num_cols + 1), dtype=int) 
a_mat_arr = np.vstack((a_mat_arr, new_row))

a_mat_arr[-1, -1] = 0

muvec_arr = np.array(mu_vector)

muvec_arr = np.append(muvec_arr, [0])

a_mat_ep_arr = np.vstack((a_mat_arr, muvec_arr))

muvec_arr = np.append(muvec_arr, [0])
a_mat_ep_arr = np.hstack((a_mat_ep_arr, np.transpose(np.asmatrix(muvec_arr))))


a_mat_ep_arr

b_vec_ep = np.zeros(num_cols, dtype=int)
b_vec_ep = np.append(b_vec_ep, [1, float(current_client_exp_ret)])

weight_ep = np.matmul(np.linalg.inv(a_mat_ep_arr),b_vec_ep)

weight_ep = weight_ep[:, :-2]

##

weight_ep_mat_transpose = np.asmatrix(weight_ep)
weight_ep_mat = np.asmatrix(np.transpose(weight_ep_mat_transpose))

ep_stddev = math.sqrt(np.matmul(np.matmul(weight_ep_mat_transpose,covariance), weight_ep_mat))
ep_exp_ret = np.matmul(mu_vector,weight_ep_mat)
ep_sharpe_ratio = (ep_exp_ret-rf_per)/ep_stddev

ep_stddev_ann = ep_stddev * math.sqrt(12)
ep_exp_ret_ann = 12 * ep_exp_ret
ep_sharpe_ratio_ann = (ep_exp_ret_ann-rf)/ep_stddev_ann

#output to excel

sheet.cell(column=1, row=18).value = 'Efficient portfolio (minimize variance) metrics:'
sheet.cell(column=2, row=18).value = 'Periodic'
sheet.cell(column=3, row=18).value = 'Annual'

sheet.cell(column=1, row=19).value = 'Expected Returns:'
sheet.cell(column=1, row=20).value = 'Standard Deviation'
sheet.cell(column=1, row=21).value = 'Sharpe Ratio'

sheet.cell(column=2, row=19).value = float(ep_exp_ret)
sheet.cell(column=2, row=20).value = float(ep_stddev)
sheet.cell(column=2, row=21).value = float(ep_sharpe_ratio)

sheet.cell(column=3, row=19).value = float(ep_exp_ret_ann)
sheet.cell(column=3, row=20).value = float(ep_stddev_ann)
sheet.cell(column=3, row=21).value = float(ep_sharpe_ratio_ann)



wb.save('MVO Assignment.xlsx')


In [193]:
#Question 3

weights_orp = np.matmul(np.linalg.inv(covariance) ,np.transpose((np.asmatrix(mu_vector) - rf_per)))
ones =  np.ones(num_cols, dtype=int)
divisor = np.matmul(ones,weights_orp)

weights_orp=weights_orp/divisor
 

weights_orp


##

weight_orp_mat = np.asmatrix(weights_orp)
weight_orp_mat_transpose = np.asmatrix(np.transpose(weight_orp_mat))

orp_stddev = math.sqrt(np.matmul(np.matmul(weight_orp_mat_transpose,covariance), weight_orp_mat))
orp_exp_ret = np.matmul(mu_vector,weight_orp_mat)
orp_sharpe_ratio = (orp_exp_ret-rf_per)/orp_stddev

orp_stddev_ann = orp_stddev * math.sqrt(12)
orp_exp_ret_ann = 12 * orp_exp_ret
orp_sharpe_ratio_ann = (orp_exp_ret_ann-rf)/orp_stddev_ann

#output to excel

sheet.cell(column=1, row=23).value = 'Optimum Risky Portfolio metrics:'
sheet.cell(column=2, row=23).value = 'Periodic'
sheet.cell(column=3, row=23).value = 'Annual'

sheet.cell(column=1, row=24).value = 'Expected Returns:'
sheet.cell(column=1, row=25).value = 'Standard Deviation'
sheet.cell(column=1, row=26).value = 'Sharpe Ratio'

sheet.cell(column=2, row=24).value = float(orp_exp_ret)
sheet.cell(column=2, row=25).value = float(orp_stddev)
sheet.cell(column=2, row=26).value = float(orp_sharpe_ratio)

sheet.cell(column=3, row=24).value = float(orp_exp_ret_ann)
sheet.cell(column=3, row=25).value = float(orp_stddev_ann)
sheet.cell(column=3, row=26).value = float(orp_sharpe_ratio_ann)

wb.save('MVO Assignment.xlsx')



In [204]:
#part 2a


max_ret_orp_line = orp_sharpe_ratio_ann*current_client_stddev_ann + rf # y = mx + b

min_sd_orp_line = (current_client_exp_ret_ann - rf) / orp_sharpe_ratio_ann # x = y-b / m

min_sd_orp_line

matrix([[0.12931784]])

In [183]:
#Question 4

weights_gmv = np.matmul(np.linalg.inv(covariance) ,np.transpose(np.asmatrix(ones)))

divisor = np.matmul(np.asmatrix(ones) ,weights_gmv)

weights_gmv = weights_gmv / divisor


weight_gmv_mat = np.asmatrix(weights_gmv)
weight_gmv_mat_transpose = np.asmatrix(np.transpose(weight_gmv_mat))


gmv_stddev = math.sqrt(np.matmul(np.matmul(weight_gmv_mat_transpose,covariance), weight_gmv_mat))
gmv_exp_ret = np.matmul(mu_vector,weight_gmv_mat)
gmv_sharpe_ratio = (gmv_exp_ret-rf_per)/gmv_stddev

gmv_stddev_ann = gmv_stddev * math.sqrt(12)
gmv_exp_ret_ann = 12 * gmv_exp_ret
gmv_sharpe_ratio_ann = (gmv_exp_ret_ann-rf)/gmv_stddev_ann

#output to excel

sheet.cell(column=1, row=13).value = 'Global Min Variance portfolio metrics:'
sheet.cell(column=2, row=13).value = 'Periodic'
sheet.cell(column=3, row=13).value = 'Annual'

sheet.cell(column=1, row=14).value = 'Expected Returns:'
sheet.cell(column=1, row=15).value = 'Standard Deviation'
sheet.cell(column=1, row=16).value = 'Sharpe Ratio'

sheet.cell(column=2, row=14).value = float(gmv_exp_ret)
sheet.cell(column=2, row=15).value = float(gmv_stddev)
sheet.cell(column=2, row=16).value = float(gmv_sharpe_ratio)

sheet.cell(column=3, row=14).value = float(gmv_exp_ret_ann)
sheet.cell(column=3, row=15).value = float(gmv_stddev_ann)
sheet.cell(column=3, row=16).value = float(gmv_sharpe_ratio_ann)



wb.save('MVO Assignment.xlsx')




In [73]:
import openpyxl
wb = openpyxl.load_workbook('MVO Assignment.xlsx', data_only=True)
sheet = wb['Output']

In [117]:
sheet.delete_cols(1, 100000)
sheet.delete_rows(1, 100000)


for i,ticker in enumerate(tickers):
    sheet.cell(column=2+i, row=2).value = ticker

sheet.cell(column=1, row=2).value = 'Ticker'



for i,mu in enumerate(mu_vector):
    sheet.cell(column=2+i, row=3).value = mu
sheet.cell(column=1, row=3).value = 'Exp. Ret.'


sheet.cell(column=2, row=4).value = current_client_stddev
sheet.cell(column=1, row=4).value = 'Std. Dev. (Client Portfolio)'

sheet.cell(column=1, row=6).value = 'Cov. Mat.'



cov_arr = np.asarray(covariance)

for i, a in enumerate(cov_arr.tolist()):
    for j, b in enumerate(a):
        sheet.cell(column=int(i+2), row=int(j+6)).value = b





# columns = history.columns.values.tolist()
# sheet.cell(column=1, row=1).value = 'Time'



# for i, time in enumerate(history.index.values.tolist()):
#     if i==0:
#         pass
#     else:
#         sheet.cell(column=1, row=i+1).value = pd.to_datetime(time)


# for i, column in enumerate(columns):
    
#     for j, value in enumerate(history[column].tolist()):
#         sheet.cell(column=i+2, row=j+1).value = value
        
# for i, column in enumerate(columns):
#     sheet.cell(column=i+2, row=1).value = column

wb.save('MVO Assignment.xlsx')


/var/folders/hg/kqg69xz568l28vnvcx6x09hh0000gn/T/ipykernel_35751/4165443610.py:4: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  covariance = np.matmul(np.transpose(history.tail(-1)),history.tail(-1))/(len(history.tail(-1)) - 1)
